<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Import Libraries</b></p> </div>

In [1]:
import numpy as np
import pandas as pd
import spacy
import string
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv') #Train Dataset
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv') #Test Dataset

In [3]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Text Prerocessing</b></p> </div>

In [5]:
from string import punctuation, digits
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
morewords=['im','youre','wer','aaa']
stop_words= stopwords+morewords
import re
def preprocess(text): 
    text = re.compile(r'https?://\S+|www\.\S+').sub('', text)#links removal
    text = text.lower() #lowercase removal
    text = re.sub(r'\d','',text) #digit removal
    text = text.strip() #white space removal
    text = re.compile(r'\b(' + '|'.join(stop_words) + r')\b\s*').sub('', text)#stopwords removal
    text=re.compile(r'[^a-zA-Z\s]').sub(r'', text)#unnecessary words Removal 
    return text

In [6]:
emoji_pattern = re.compile("["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+", flags=re.UNICODE)

In [7]:
train_df['pre_text'] = train_df["text"].apply(preprocess) # calling the preprocess for cleaning the data
train_df['pre_text'] = train_df['pre_text'].apply(lambda x: emoji_pattern.sub(r'',x)) # remove emojies from the text

In [8]:
test_df['pre_text'] = test_df["text"].apply(preprocess) # calling the preprocess for cleaning the data
test_df['pre_text'] = test_df['pre_text'].apply(lambda x: emoji_pattern.sub(r'',x)) # remove emojies from the text

In [9]:
train_df[['text','pre_text']]

,text,pre_text
0,Our Deeds are the Reason of this #earthquake M...,deeds reason earthquake may allah forgive us
1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...
3,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders ca...
4,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfires pou...
...,...,...
7608,Two giant cranes holding a bridge collapse int...,two giant cranes holding bridge collapse nearb...
7609,@aria_ahrary @TheTawniest The out of control w...,ariaahrary thetawniest control wild fires cali...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,utckm volcano hawaii
7611,Police investigating after an e-bike collided ...,police investigating ebike collided car little...


In [10]:
test_df[['text','pre_text']]

,text,pre_text
0,Just happened a terrible car crash,happened terrible car crash
1,"Heard about #earthquake is different cities, s...",heard earthquake different cities stay safe ev...
2,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...
3,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety los angeles safety fastener...
3259,Storm in RI worse than last hurricane. My city...,storm ri worse last hurricane cityampothers ha...
3260,Green Line derailment in Chicago http://t.co/U...,green line derailment chicago
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issues hazardous weather outlook hwo


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Split the data</b></p> </div>

In [11]:
x = train_df['pre_text']
y = train_df['target']

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [13]:
print(len(y_train),len(x_train),len(x_test),len(y_test))

6090 6090 1523 1523


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Make vacabulary of Data</b></p> </div>

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [15]:
vocab_size = 1000
oov_tok = '<OOV>'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([str(sample) for sample in x_train])
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'like': 2,
 'amp': 3,
 'fire': 4,
 'get': 5,
 'via': 6,
 'new': 7,
 'news': 8,
 'people': 9,
 'one': 10}

<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Convert Text sentences into Numeric Vector by using Vocabulary </b></p> </div>

In [16]:
train_sequences = tokenizer.texts_to_sequences(x_train)
print(train_sequences[0])

[1, 1, 1, 853, 1]


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Make sentances in Equal Length</b></p> </div>

In [17]:
embedding_dim = 32
max_length = 50
trunc_type = 'post'  # remove or truncate last words in sentences if max_length > 50 ans "post" defined last at sentence
padding_type = 'post'

In [18]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))
print(len(x_train[0]))

5
50
45


In [19]:
validation_sequences = tokenizer.texts_to_sequences(x_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

1523
(1523, 50)


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Model Building</b></p> </div>

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.LSTM(32,activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          32000     
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 41,409
Trainable params: 41,409
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 5
history = model.fit(train_padded, y_train, epochs=num_epochs, validation_data=(validation_padded, y_test), verbose=2)

Epoch 1/5
191/191 - 7s - loss: 0.6852 - accuracy: 0.5691 - val_loss: 0.6830 - val_accuracy: 0.5712 - 7s/epoch - 35ms/step
Epoch 2/5
191/191 - 4s - loss: 0.6809 - accuracy: 0.5749 - val_loss: 0.6830 - val_accuracy: 0.5712 - 4s/epoch - 21ms/step
Epoch 3/5
191/191 - 4s - loss: 0.6835 - accuracy: 0.5701 - val_loss: 0.6833 - val_accuracy: 0.5712 - 4s/epoch - 22ms/step
Epoch 4/5
191/191 - 4s - loss: 0.6837 - accuracy: 0.5701 - val_loss: 0.6832 - val_accuracy: 0.5712 - 4s/epoch - 21ms/step
Epoch 5/5
191/191 - 4s - loss: 0.6837 - accuracy: 0.5701 - val_loss: 0.6830 - val_accuracy: 0.5712 - 4s/epoch - 21ms/step


<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Save the model</b></p> </div>

In [22]:
model.save('model.h5')

In [23]:
from keras.models import load_model
model = load_model('model.h5')

<div style="color:white;display:fill;border-radius:8px; background-color:#323232;font-size:150%; font-family:Nexa;letter-spacing:0.5px"> <p style="padding: 8px;color:white;"><b>Model on testing file</b></p> </div>

In [24]:
validation_sequences= tokenizer.texts_to_sequences(test_df['pre_text'])
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

3263
(3263, 50)


In [25]:
y_pred = model.predict(validation_padded)

102/102 [==============================] - 1s 7ms/step


In [26]:
target = [1 if pred > 0.5 else 0 for pred in y_pred]

In [27]:
test1=test_df['id']
test1

0           0
1           2
2           3
3           9
4          11
        ...  
3258    10861
3259    10865
3260    10868
3261    10874
3262    10875
Name: id, Length: 3263, dtype: int64

In [28]:
Predict = pd.DataFrame({
    "id":test1.values,
    "target" : target
},columns=["id","target"])
Predict

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [29]:
Predict.set_index("id", inplace=True)
Predict.to_csv("sample_submission.csv")

In [30]:
pd.read_csv("sample_submission.csv")

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0
